# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [5]:
pd.set_option('display.max_colwidth', None)

In [7]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("clean_disaster_data", engine)

In [14]:
X = df["message"]
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

In [12]:
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
        ('tfidf_transformer', TfidfTransformer())
    ])) ,
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)

c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\luo

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
from sklearn.metrics import classification_report

In [17]:
y_prediction_train = pipeline_fitted.predict(X_test)

In [18]:
for idx, column in enumerate(y.columns.values):
    print("Target: ", column)
    print(classification_report(y_test[column], y_prediction_train[:, idx]))
    print("---" * 10)

Target:  related
              precision    recall  f1-score   support

           0       0.58      0.24      0.34      1527
           1       0.80      0.95      0.87      4984
           2       0.29      0.14      0.19        43

    accuracy                           0.78      6554
   macro avg       0.56      0.44      0.46      6554
weighted avg       0.75      0.78      0.74      6554

------------------------------
Target:  request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5432
           1       0.77      0.55      0.64      1122

    accuracy                           0.89      6554
   macro avg       0.84      0.76      0.79      6554
weighted avg       0.89      0.89      0.89      6554

------------------------------
Target:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy     

### 6. Improve your model

Add another feature named __StartingVerb__

In [17]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

Create a new pipeline which union this new feature before putting it into the classifier algorithm

In [18]:
pipeline_new = Pipeline([
    ('feature', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor()) 
    ])) ,
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

Use grid search to find better parameters.

In [21]:
parameters =  {
    'classifier__estimator__learning_rate': [0.5, 1.0],
    'classifier__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline_new, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 

cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\luong\anaconda3\envs\analysis\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\luo

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000020B249A5940>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.5, 1.0],
                         'classifier__estimator__n_estimators': [10, 20]},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_prediction_train = cv.predict(X_test)

In [46]:
for idx, column in enumerate(y.columns.values):
    print("Target: ", column)
    print(classification_report(y_test[column], y_prediction_train[:, idx]))
    print("---" * 10)

Target:  related
              precision    recall  f1-score   support

           0       0.55      0.19      0.28      4577
           1       0.79      0.95      0.86     14954
           2       0.38      0.07      0.12       131

    accuracy                           0.77     19662
   macro avg       0.57      0.40      0.42     19662
weighted avg       0.73      0.77      0.72     19662

------------------------------
Target:  request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     16348
           1       0.77      0.50      0.61      3314

    accuracy                           0.89     19662
   macro avg       0.84      0.74      0.77     19662
weighted avg       0.88      0.89      0.88     19662

------------------------------
Target:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19565
           1       0.33      0.03      0.06        97

    accuracy     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [47]:
import pickle

In [48]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.